In [2]:
import findspark


In [3]:
findspark.init('/home/admin123/spark-2.4.0-bin-hadoop2.7/')

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('LReg').getOrCreate()

In [6]:
from pyspark.ml.regression import LinearRegression

In [7]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [8]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [9]:
# Linear Regression takes three parameters featureCol, labelCol, and a predictionCol and also more other but currently use only these
lr =LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction' )

In [10]:
lrModel = lr.fit(training)

In [11]:
#Coefficients corresponds to a particular fearture
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [12]:
lrModel.intercept

0.14228558260358093

In [13]:
training_summary = lrModel.summary

In [14]:
# R2 explains how much variance explains by your model
training_summary.r2

0.027839179518600154

In [15]:
training_summary.rootMeanSquaredError

10.16309157133015

In [16]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [19]:
#splitting data into training and testing data
training_data,test_data = all_data.randomSplit([0.7, 0.3])

In [20]:
training_data

DataFrame[label: double, features: vector]

In [21]:
test_data

DataFrame[label: double, features: vector]

In [25]:
#show training data after splitting
training_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                347|
|   mean|0.14696310236105975|
| stddev| 10.094506488426353|
|    min|-26.805483428483072|
|    max|  27.78383192005107|
+-------+-------------------+



In [26]:
#show test data after splitting
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                154|
|   mean| 0.5045785935091059|
| stddev| 10.834057073101919|
|    min|-28.571478869743427|
|    max|  22.31738046492344|
+-------+-------------------+



In [27]:
 correct_modal = lr.fit(training_data)

In [28]:
test_results = correct_modal.evaluate(test_data)

In [31]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-27.417902720081425|
| -27.22388317339978|
|-23.903079474975254|
|-25.516600447083118|
|-20.750561048611768|
|-19.862316670487367|
|-19.827082471968804|
|-18.748428428897217|
|-18.006945354255702|
|-15.532928797544624|
|-16.074136120988207|
|-15.220568406411454|
|-16.957723233714567|
| -13.43583061422257|
|-13.091343061051756|
| -13.67678129349798|
|  -16.3158000263706|
|-13.275426255867185|
|-14.034518801322644|
|-12.154187911054622|
+-------------------+
only showing top 20 rows



In [33]:
test_results.rootMeanSquaredError

10.597389498949694

In [34]:
# Applying Model on un labeled data 
unlabeled_data = test_data.select('features')


In [35]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [37]:
# to predict housing price by unlabeled data
#passing unlabeld data to trained model predictions

predictions = correct_modal.transform(unlabeled_data)

In [38]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -1.1535761496620005|
|(10,[0,1,2,3,4,5,...| -0.8221348643768529|
|(10,[0,1,2,3,4,5,...|   -2.83312770762647|
|(10,[0,1,2,3,4,5,...|   2.566774510887045|
|(10,[0,1,2,3,4,5,...|  0.6930784328225561|
|(10,[0,1,2,3,4,5,...| 0.07955388087282991|
|(10,[0,1,2,3,4,5,...| 0.15976385659708464|
|(10,[0,1,2,3,4,5,...|  1.3197538579577104|
|(10,[0,1,2,3,4,5,...|  1.3147383329445963|
|(10,[0,1,2,3,4,5,...|-0.41858376824994903|
|(10,[0,1,2,3,4,5,...|  0.3506205079396381|
|(10,[0,1,2,3,4,5,...|-0.21681638701976347|
|(10,[0,1,2,3,4,5,...|    1.59817835388189|
|(10,[0,1,2,3,4,5,...|  -1.913040541156683|
|(10,[0,1,2,3,4,5,...|  -2.219637528364533|
|(10,[0,1,2,3,4,5,...| -1.0859769594331474|
|(10,[0,1,2,3,4,5,...|  2.5433584646677287|
|(10,[0,1,2,3,4,5,...| 0.23549819176256986|
|(10,[0,1,2,3,4,5,...|   1.543076723776232|
|(10,[0,1,2,3,4,5,...|  0.249201